In [14]:
from sympy import Matrix, symbols
from IPython.display import display, Markdown

class SymbolicMatrix:
    def __init__(self, matrix, column_labels=None, free_terms=None):
        """
        Inicjalizuje macierz, z opcjonalnymi nazwami kolumn i wyrazami wolnymi.
        
        Parameters:
        - matrix: Macierz główna.
        - column_labels: Lista etykiet kolumn (opcjonalnie).
        - free_terms: Lista wyrazów wolnych (opcjonalnie).
        """
        self.matrix = Matrix(matrix)
        self.operations = []
        
        # Przechowuje etykiety kolumn, jeśli dostarczono
        if column_labels:
            if len(column_labels) != self.matrix.cols:
                raise ValueError("Liczba etykiet kolumn musi być równa liczbie kolumn w macierzy.")
            self.column_labels = column_labels
        else:
            self.column_labels = [f"x{i+1}" for i in range(self.matrix.cols)]
        
        # Przechowuje wyrazy wolne, jeśli dostarczono
        if free_terms:
            if len(free_terms) != self.matrix.rows:
                raise ValueError("Liczba wyrazów wolnych musi być równa liczbie wierszy w macierzy.")
            self.free_terms = free_terms
        else:
            self.free_terms = [0 for _ in range(self.matrix.rows)]
        
        display(Markdown("**Początkowa macierz:**"))
        self.display_matrix()

    def __repr__(self):
        return repr(self.matrix)

    def __str__(self):
        return str(self.matrix)

    def _repr_latex_(self):
        return self.matrix._repr_latex_()

    def _validate_row_number(self, row):
        if not isinstance(row, int):
            raise TypeError("Numer wiersza musi być liczbą całkowitą.")
        if row < 1 or row > self.matrix.rows:
            raise IndexError(f"Numer wiersza musi być w zakresie od 1 do {self.matrix.rows}.")
        return row - 1

    def _validate_col_number(self, col):
        if not isinstance(col, int):
            raise TypeError("Numer kolumny musi być liczbą całkowitą.")
        if col < 1 or col > self.matrix.cols:
            raise IndexError(f"Numer kolumny musi być w zakresie od 1 do {self.matrix.cols}.")
        return col - 1

    def add_row(self, target_row, source_row, coefficient):
        target_idx = self._validate_row_number(target_row)
        source_idx = self._validate_row_number(source_row)
        
        self.matrix.row_op(target_idx, lambda v, j: v + coefficient * self.matrix[source_idx, j])
        
        operation_str = f"w{target_row} = w{target_row} + {coefficient}*w{source_row}"
        self.operations.append(operation_str)
        display(Markdown(f"**Operacja:** {operation_str}"))
        self.display_matrix()

    def multiply_row(self, row, coefficient):
        row_idx = self._validate_row_number(row)
        
        self.matrix.row_op(row_idx, lambda v, _: coefficient * v)
        
        operation_str = f"w{row} = {coefficient}*w{row}"
        self.operations.append(operation_str)
        display(Markdown(f"**Operacja:** {operation_str}"))
        self.display_matrix()

    def swap_rows(self, row1, row2):
        row1_idx = self._validate_row_number(row1)
        row2_idx = self._validate_row_number(row2)
        
        self.matrix.row_swap(row1_idx, row2_idx)
        self.free_terms[row1_idx], self.free_terms[row2_idx] = self.free_terms[row2_idx], self.free_terms[row1_idx]
        
        operation_str = f"Zamiana w{row1} <-> w{row2}"
        self.operations.append(operation_str)
        display(Markdown(f"**Operacja:** {operation_str}"))
        self.display_matrix()

    def swap_columns(self, col1, col2):
        col1_idx = self._validate_col_number(col1)
        col2_idx = self._validate_col_number(col2)
        
        self.matrix.col_swap(col1_idx, col2_idx)
        self.column_labels[col1_idx], self.column_labels[col2_idx] = self.column_labels[col2_idx], self.column_labels[col1_idx]
        
        operation_str = f"Zamiana kol{col1} <-> kol{col2}"
        self.operations.append(operation_str)
        display(Markdown(f"**Operacja:** {operation_str}"))
        self.display_matrix()

    def display_matrix(self):
        """Wyświetla macierz z etykietami kolumn i wyrazami wolnymi."""
        matrix_with_free_terms = self.matrix.row_join(Matrix(self.free_terms))
        headers = self.column_labels + ["Wyraz wolny"]
        display(Markdown(self._matrix_to_markdown(matrix_with_free_terms, headers)))

    def _matrix_to_markdown(self, matrix, headers):
        """Konwertuje macierz do formatu Markdown z etykietami."""
        markdown_str = "| " + " | ".join(headers) + " |\n"
        markdown_str += "| " + " | ".join(["---"] * len(headers)) + " |\n"
        for i in range(matrix.rows):
            row = [str(matrix[i, j]) for j in range(matrix.cols)]
            markdown_str += "| " + " | ".join(row) + " |\n"
        return markdown_str

    def print_operations(self):
        display(Markdown("**Wykonane operacje:**"))
        for op in self.operations:
            print(op)

# Przykład użycia:
m = SymbolicMatrix([[1, 2, 3], [4, 5, 6], [7, 8, 10]], column_labels=["x1", "x2", "x3"], free_terms=[10, 11, 12])



**Początkowa macierz:**

| x1 | x2 | x3 | Wyraz wolny |
| --- | --- | --- | --- |
| 1 | 2 | 3 | 10 |
| 4 | 5 | 6 | 11 |
| 7 | 8 | 10 | 12 |


In [15]:
m.add_row(2, 1, -4)

**Operacja:** w2 = w2 + -4*w1

| x1 | x2 | x3 | Wyraz wolny |
| --- | --- | --- | --- |
| 1 | 2 | 3 | 10 |
| 0 | -3 | -6 | 11 |
| 7 | 8 | 10 | 12 |


In [16]:
m.add_row(3, 1, -7)

**Operacja:** w3 = w3 + -7*w1

| x1 | x2 | x3 | Wyraz wolny |
| --- | --- | --- | --- |
| 1 | 2 | 3 | 10 |
| 0 | -3 | -6 | 11 |
| 0 | -6 | -11 | 12 |


In [17]:
m.add_row(3, 2, -2)

**Operacja:** w3 = w3 + -2*w2

| x1 | x2 | x3 | Wyraz wolny |
| --- | --- | --- | --- |
| 1 | 2 | 3 | 10 |
| 0 | -3 | -6 | 11 |
| 0 | 0 | 1 | 12 |
